In [67]:
import random
import string
import time
import copy
import sys

In [68]:
def random_board(N):
       return [ random.randint(1, N) for _ in range(N) ]

In [69]:
def heuristic_value(individual):
    horizontal_collisions = sum([individual.count(queen)-1 for queen in individual])/2
    diagonal_collisions = 0

    n = len(individual)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        left_diagonal[i + individual[i] - 1] += 1
        right_diagonal[len(individual) - i + individual[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]-1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]-1
        diagonal_collisions += counter #/ (n-abs(i-n+1))
    
    return int(horizontal_collisions + diagonal_collisions)

In [70]:
# https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python
# https://stackoverflow.com/questions/44430194/roulette-wheel-selection-with-positive-and-negative-fitness-values-for-minimizat
# Weighted random choice where a board is likelier to be chosen the lower its heuristic value is 

def random_pick(board_choices, board_weights):
    
    board_weights = [1/w for w in board_weights]
    
    
    boardWithWeight = zip(board_choices,board_weights)
    
    total = sum(w for c,w in boardWithWeight)
    #board_weights = [w/total for w in board_weights]
    
    r = random.uniform(0,total)
    upto = 0
    for c, w in zip(board_choices, board_weights):
        if upto + w >= r:
            return c
        upto += w

In [103]:
# If the board_choices list already contain a solution to the N Queens Problem, return the solution board.
def pick_best(board_choices, board_weights):
    for c, w in zip(board_choices, board_weights):
        if w == 0:
            return c


In [104]:
def hill_climbing(board):
    # Choose a random board with a better heuristic value

    min_board = board
    min_h = heuristic_value(board)
    global n_steps, stuck

    n_steps += 1


    board_choices = []
    board_weights = []
    '''
    Find the boards with better heuristic value than the current board, and keep the boards and their 
    heuristic values in lists.
    '''
    for i in range(N):
        # Find index of queen in current row
        queen = board[i]
        k=0
        for k in range (N):
            if k != board[i]-1:
                board[i] = k+1

                h = heuristic_value(board)

                if h < min_h:
                    board_weights.append(h)
                    temp_board = copy.deepcopy(board)
                    #print(temp_board)
                    board_choices.append(temp_board)
        board[i] = queen
    '''
    If board_choices is non-empty, there are neighbour boards with better heuristic values than the current board.
    Select one board to be the new min_board.
    
    If board_choices is empty, there are no neighbour boards with better heuristic 
    '''
    if board_choices:
        if 0 not in board_weights:
            new_board = random_pick(board_choices,board_weights)
        else: new_board = pick_best(board_choices,board_weights)
        
        min_h = heuristic_value(new_board)
        min_board = new_board
        return min_board
        
    else: 
        stuck = True
        return min_board   
        

In [108]:
if __name__ == "__main__":
    N = input("Enter your value: ")
    N = int(N)
    
    board = random_board(N)

    start_time = time.time()
    
    #n_side_moves = 0
    n_steps = 0

    
    #print("Current position's heuristic value: ", heuristic_value(board))
    stuck = False 
    while not stuck:
        if heuristic_value(board)==0:
            print("Final Heuristic value: ",heuristic_value(min_board))
            print("Number of steps: ",n_steps)
            break
            
        else: 
            min_board = hill_climbing(board)
            print("Current board: ", min_board)
            print("Current position's heuristic value: ", heuristic_value(min_board))
            board = min_board


    if heuristic_value(min_board)== 0:
           print (min_board)

    else:
        print(min_board)
        print("Could not solve")

    print("%s seconds" % (time.time() - start_time))


Enter your value: 5
Current board:  [2, 2, 3, 2, 5]
Current position's heuristic value:  6
Current board:  [2, 1, 3, 2, 5]
Current position's heuristic value:  4
Current board:  [4, 1, 3, 2, 5]
Current position's heuristic value:  2
Current board:  [4, 1, 4, 2, 5]
Current position's heuristic value:  1
Current board:  [3, 1, 4, 2, 5]
Current position's heuristic value:  0
Final Heuristic value:  0
Number of steps:  5
[3, 1, 4, 2, 5]
0.014782428741455078 seconds
